In [1]:
import pywraps2 as s2
import json
import collections
import logging as logger
import copy
from collections import OrderedDict
import time
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import pyproj as proj

In [2]:
crs_wgs = proj.Proj(init='epsg:4326') # assuming you're using WGS84 geographic
crs_bng = proj.Proj(init='epsg:32627') # use a locally appropriate projected CRS

In [3]:
logger.basicConfig(format='   %(message)s', level=logger.DEBUG)

In [4]:
bounding_local_parsed_path = 'bounding_local_parsed_final.json'
bounding_university_parsed_path = 'bounding_university_parsed_final.json'
bounding_college_parsed_path = 'bounding_college_parsed_final.json'
bad_bounding_local_parsed_path = 'bounding_local_parsed_bad.json'
bad_bounding_university_parsed_path = 'bounding_university_parsed_bad.json'
bad_bounding_college_parsed_path = 'bounding_college_parsed_bad.json'

In [5]:
with open(bounding_local_parsed_path, 'r') as read_file:
    bounding_local = json.load(read_file)
with open(bounding_university_parsed_path, 'r') as read_file:
    bounding_university = json.load(read_file)
with open(bounding_college_parsed_path, 'r') as read_file:
    bounding_college = json.load(read_file)
with open(bad_bounding_local_parsed_path, 'r') as read_file:
    bad_bounding_local = json.load(read_file)
with open(bad_bounding_university_parsed_path, 'r') as read_file:
    bad_bounding_university = json.load(read_file)
with open(bad_bounding_college_parsed_path, 'r') as read_file:
    bad_bounding_college = json.load(read_file)

In [6]:
def findname(alist, name):
    alist_copy = copy.deepcopy(alist)
    for bindex, bound in enumerate(alist_copy):
        if bound['name'] == name:
            logger.debug(bindex)

In [7]:
def findingbadpoints(alist, badlist):
    alist_copy = copy.deepcopy(alist)
    #startindex = badlist[-1]
    startindex = -1
    for progindex, bound in enumerate(alist_copy[startindex + 1:]):
        logger.debug(progindex + startindex + 1)
        if progindex in badlist:
            continue
        if bound['type'] == 'relation':
            for waylist in bound['nodelist']:
                while waylist[0] == waylist[-1]:
                    waylist.pop(-1)
                for pairindex, pair in enumerate(waylist):
                    oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                    newpoint = oldlatlng.ToPoint().Normalize()
                    waylist[pairindex] = newpoint
                testpoly_area_original = s2.S2Polygon(s2.S2Loop(waylist)).GetArea()
                testpoly_area_reverse = s2.S2Polygon(s2.S2Loop(waylist[::-1])).GetArea()
                if testpoly_area_reverse < testpoly_area_original:
                    waylist = waylist[::-1]
        if bound['type'] == 'way':
            waylist = bound['nodelist']
            while waylist[0] == waylist[-1]:
                waylist.pop(-1)
            for pairindex, pair in enumerate(waylist):
                oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                newpoint = oldlatlng.ToPoint().Normalize()
                waylist[pairindex] = newpoint
            testpoly_area_original = s2.S2Polygon(s2.S2Loop(waylist)).GetArea()
            testpoly_area_reverse = s2.S2Polygon(s2.S2Loop(waylist[::-1])).GetArea()
            if testpoly_area_reverse < testpoly_area_original:
                waylist = waylist[::-1]
    return alist_copy

In [8]:
bounding_university_badlist = [514,1296,1378,1812,1917,2285,2411,2435,6110,10645,12401,15141,16536,18674,
                              18930,20253,21126,21349,23327,25376,25494,28016,28540,29194,29733,30224,31256,32139,
                              32508,32740,32960,33163,33235,35133,35384,35493,35501,35503,35506,35512,35516,35522]
bounding_college_badlist = [916,5511,11284,11456,12991,14602,21016,21935,25595,26339,26364,26444,26697,27759,28929,
                           29081,30192,30195]


In [9]:
def filtergoods(alist,badlist):
    alist_copy = copy.deepcopy(alist)
    bads = []
    goods = []
    for progindex, bound in enumerate(alist_copy):
        if progindex in badlist:
            bads.append(bound)
        else:
            goods.append(bound)
    return goods, bads

In [10]:
filtered_bounding_university, not_looping_university = filtergoods(bounding_university, bounding_university_badlist)
filtered_bounding_college, not_looping_college = filtergoods(bounding_college, bounding_college_badlist)

In [14]:
def correctlyorientlist(alist):
    alist_copy = copy.deepcopy(alist)
    for progindex, bound in enumerate(alist_copy):
        #logger.debug(progindex)
        if bound['type'] == 'relation':
            for windex,waylist in enumerate(bound['nodelist']):
                waylistcopy = copy.deepcopy(waylist)
                waylisttestcopy = copy.deepcopy(waylist)
                while waylisttestcopy[0] == waylisttestcopy[-1]:
                    waylisttestcopy.pop(-1)
                for pairindex, pair in enumerate(waylisttestcopy):
                    oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                    newpoint = oldlatlng.ToPoint().Normalize()
                    waylisttestcopy[pairindex] = newpoint
                testpoly_area_original = s2.S2Polygon(s2.S2Loop(waylisttestcopy)).GetArea()
                testpoly_area_reverse = s2.S2Polygon(s2.S2Loop(waylisttestcopy[::-1])).GetArea()
                #for some reason its greater than despite the logic being backwards
                if testpoly_area_reverse < testpoly_area_original:
                    bound['nodelist'][windex] = waylistcopy[::-1]
        if bound['type'] == 'way':
            waylist = bound['nodelist']
            waylistcopy = copy.deepcopy(waylist)
            waylisttestcopy = copy.deepcopy(waylist)
            while waylisttestcopy[0] == waylisttestcopy[-1]:
                waylisttestcopy.pop(-1)
            for pairindex, pair in enumerate(waylisttestcopy):
                oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                newpoint = oldlatlng.ToPoint().Normalize()
                waylisttestcopy[pairindex] = newpoint
            testpoly_area_original = s2.S2Polygon(s2.S2Loop(waylisttestcopy)).GetArea()
            testpoly_area_reverse = s2.S2Polygon(s2.S2Loop(waylisttestcopy[::-1])).GetArea()
            #for some reason its greater than despite the logic being backwards
            if testpoly_area_reverse < testpoly_area_original:
                bound['nodelist'] = waylistcopy[::-1]
            
    return alist_copy

In [15]:
oriented_bounding_university = correctlyorientlist(filtered_bounding_university)
oriented_bounding_college = correctlyorientlist(filtered_bounding_college)
oriented_bounding_local = correctlyorientlist(bounding_local)

In [16]:
with open(bounding_university_parsed_path[:-5] + '_cleaned.json', 'w') as write_file:
    json.dump(oriented_bounding_university,write_file)
with open(bounding_college_parsed_path[:-5] + '_cleaned.json', 'w') as write_file:
    json.dump(oriented_bounding_college,write_file)   
with open(bounding_local_parsed_path[:-5] + '_cleaned.json', 'w') as write_file:
    json.dump(oriented_bounding_local,write_file)
    
with open(bounding_university_parsed_path[:-5] + '_noloops.json', 'w') as write_file:
    json.dump(not_looping_university,write_file)
with open(bounding_college_parsed_path[:-5] + '_noloops.json', 'w') as write_file:
    json.dump(not_looping_college,write_file)


In [11]:
def adjusttos2points(alist):
    alist_copy = copy.deepcopy(alist)
    for progindex, bound in enumerate(alist_copy):
        #logger.debug(progindex)
        if bound['type'] == 'relation':
            for windex,waylist in enumerate(bound['nodelist']):
                while waylist[0] == waylist[-1]:
                    waylist.pop(-1)
                for pairindex, pair in enumerate(waylist):
                    oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                    newpoint = oldlatlng.ToPoint().Normalize()
                    waylist[pairindex] = newpoint
                testpoly_area_original = s2.S2Polygon(s2.S2Loop(waylist)).GetArea()
                testpoly_area_reverse = s2.S2Polygon(s2.S2Loop(waylist[::-1])).GetArea()
                #for some reason its greater than despite the logic being backwards
                if testpoly_area_reverse < testpoly_area_original:
                    bound['nodelist'][windex] = waylist[::-1]
        if bound['type'] == 'way':
            waylist = bound['nodelist']
            while waylist[0] == waylist[-1]:
                waylist.pop(-1)
            for pairindex, pair in enumerate(waylist):
                oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                newpoint = oldlatlng.ToPoint().Normalize()
                waylist[pairindex] = newpoint
            testpoly_area_original = s2.S2Polygon(s2.S2Loop(waylist)).GetArea()
            testpoly_area_reverse = s2.S2Polygon(s2.S2Loop(waylist[::-1])).GetArea()
            #for some reason its greater than despite the logic being backwards
            if testpoly_area_reverse < testpoly_area_original:
                bound['nodelist'] = waylist[::-1]
            
    return alist_copy

In [12]:
points_university = adjusttos2points(filtered_bounding_university)
points_college = adjusttos2points(filtered_bounding_college)
points_local = adjusttos2points(filtered_bounding_local)

In [13]:
def pointstos2cells(alist):
    cover16s = s2.S2RegionCoverer()
    cover16s.set_fixed_level(16)
    cover10s = s2.S2RegionCoverer()
    cover10s.set_fixed_level(10)
    coverl10_l11 = s2.S2RegionCoverer()
    coverl10_l11.set_max_level(11)
    coverl10_l11.set_min_level(10)
    coverl10tol16 = s2.S2RegionCoverer()
    coverl10tol16.set_max_level(16)
    coverl10tol16.set_min_level(10)
    for progindex, bound in enumerate(alist):
        logger.debug(progindex)
        if bound['type'] == 'relation':
            looplist = []
            areas = []
            for waylist in bound['nodelist']:
                wloop = s2.S2Loop(waylist)
                wpoly = s2.S2Polygon(wloop)
                looplist.append(wloop)
                areas.append(wpoly.GetArea())
            newareas, looplist = zip(*sorted(zip(areas, looplist)))
            logger.debug(newareas)
            looplist = list(looplist)
            using_polygon = s2.S2Polygon()
            using_polygon.InitNested(looplist)
            coveringl10_to_l16 = map(lambda x: x.ToToken(),coverl10tol16.GetCovering(wloop))
            coveringl10s = map(lambda x: x.ToToken(),cover10s.GetCovering(wloop))
            coveringl10_and_l11 = map(lambda x: x.ToToken(),coverl10_l11.GetCovering(wloop))
            coveringl16s = map(lambda x: x.ToToken(),cover16s.GetCovering(wloop))

            bound['cl10s'] = coveringl10s
            bound['cl10tol16'] = coveringl10_to_l16
            bound['cl10+l11'] = coveringl10_and_l11
            bound['cl16s'] = coveringl16s
        if bound['type'] == 'way':
            using_polygon = s2.S2Polygon(s2.S2Loop(bound['nodelist']))
            coveringl10_to_l16 = map(lambda x: x.ToToken(),coverl10tol16.GetCovering(using_polygon))
            coveringl10s = map(lambda x: x.ToToken(),cover10s.GetCovering(using_polygon))
            coveringl10_and_l11 = map(lambda x: x.ToToken(),coverl10_l11.GetCovering(using_polygon))
            coveringl16s = map(lambda x: x.ToToken(),cover16s.GetCovering(using_polygon))
            
            bound['cl10s'] = coveringl10s
            bound['cl10tol16'] = coveringl10_to_l16
            bound['cl10+l11'] = coveringl10_and_l11
            bound['cl16s'] = coveringl16s
    #return alist_copy

In [ ]:
s2_university = pointstos2cells(points_university)
s2_college = pointstos2cells(points_college)
s2_local = pointstos2cells(points_local)

   0
   (5.418522060438454e-11, 1.6938382448817692e-10)


In [11]:
def borderstos2cells(alist):
    alist_copy = copy.deepcopy(alist)
    for progindex, bound in enumerate(alist_copy):
        cover16s = s2.S2RegionCoverer()
        cover16s.set_fixed_level(16)
        cover10s = s2.S2RegionCoverer()
        cover10s.set_fixed_level(10)
        coverl10_l11 = s2.S2RegionCoverer()
        coverl10_l11.set_max_level(11)
        coverl10_l11.set_min_level(10)
        coverl10tol16 = s2.S2RegionCoverer()
        coverl10tol16.set_max_level(16)
        coverl10tol16.set_min_level(10)
        logger.debug(progindex)
        if bound['type'] == 'relation':
            looplist = []
            for windex,waylist in enumerate(bound['nodelist']):
                while waylist[0] == waylist[-1]:
                    waylist.pop(-1)
                for pairindex, pair in enumerate(waylist):
                    oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                    newpoint = oldlatlng.ToPoint().Normalize()
                    waylist[pairindex] = newpoint
                testloop_original = s2.S2Loop(waylist)
                testloop_reverse = s2.S2Loop(waylist[::-1])
                testpoly_original = s2.S2Polygon(testloop_original)
                testpoly_reverse = s2.S2Polygon(testloop_reverse)
                testpoly_area_original = testpoly_original.GetArea()
                testpoly_area_reverse = testpoly_reverse.GetArea()
                if testpoly_area_reverse < testpoly_area_original:
                    looplist.append(testloop_reverse)
                else:
                    looplist.append(testloop_original)
            logger.debug(looplist)
            using_polygon = s2.S2Polygon()
            using_polygon.InitNested(looplist)
            logger.debug('initialized polygon')
            coveringl10_to_l16 = map(lambda x: x.ToToken(),coverl10tol16.GetCovering(using_polygon))
            coveringl10s = map(lambda x: x.ToToken(),cover10s.GetCovering(using_polygon))
            coveringl10_and_l11 = map(lambda x: x.ToToken(),coverl10_l11.GetCovering(using_polygon))
            coveringl16s = map(lambda x: x.ToToken(),cover16s.GetCovering(using_polygon))
            
            bound['cl10s'] = coveringl10s
            bound['cl10tol16'] = coveringl10_to_l16
            bound['cl10+l11'] = coveringl10_and_l11
            bound['cl16s'] = coveringl16s
        if bound['type'] == 'way':
            waylist = bound['nodelist']
            while waylist[0] == waylist[-1]:
                waylist.pop(-1)
            for pairindex, pair in enumerate(waylist):
                oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                newpoint = oldlatlng.ToPoint().Normalize()
                waylist[pairindex] = newpoint
                
            testpoly_original = s2.S2Polygon(s2.S2Loop(waylist))
            testpoly_reverse = s2.S2Polygon(s2.S2Loop(waylist[::-1]))
            testpoly_area_original = testpoly_original.GetArea()
            testpoly_area_reverse = testpoly_reverse.GetArea()
            using_polygon = testpoly_original
            if testpoly_area_reverse < testpoly_area_original:
                using_polygon = testpoly_reverse
            coveringl10_to_l16 = map(lambda x: x.ToToken(),coverl10tol16.GetCovering(using_polygon))
            coveringl10s = map(lambda x: x.ToToken(),cover10s.GetCovering(using_polygon))
            coveringl10_and_l11 = map(lambda x: x.ToToken(),coverl10_l11.GetCovering(using_polygon))
            coveringl16s = map(lambda x: x.ToToken(),cover16s.GetCovering(using_polygon))
            
            bound['cl10s'] = coveringl10s
            bound['cl10tol16'] = coveringl10_to_l16
            bound['cl10+l11'] = coveringl10_and_l11
            bound['cl16s'] = coveringl16s
    return alist_copy

In [ ]:
s2_university = borderstos2cells(points_university)
s2_college = borderstos2cells(points_college)
s2_local = borderstos2cells(points_local)

   0
   [<pywraps2.S2Loop; proxy of <Swig Object of type 'S2Loop *' at 0x12f8b1ba0> >, <pywraps2.S2Loop; proxy of <Swig Object of type 'S2Loop *' at 0x12f8b1690> >]


In [3]:
allloops = [s2.S2Loop(element) for element in points_university[906]['nodelist']]

NameError: name 'points_university' is not defined

In [20]:
polygon = s2.S2Polygon()

In [21]:
polygon.InitNested(allloops)

In [22]:
polygon.GetArea()

1.0394898163350016e-07

In [ ]:
s2.S2

In [14]:
pb = s2.S2Polygon()

In [15]:
pb.PolygonBuilder()

AttributeError: 'S2Polygon' object has no attribute 'PolygonBuilder'

In [13]:
len(points_university[906]['nodelist'])

25

In [14]:
test1 = points_university[906]['nodelist'][1]

In [15]:
test1

[<pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5de0> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5db0> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5d80> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5d50> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5d20> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5cf0> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5cc0> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5c90> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5c60> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5c30> >,
 <pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x1316b5c00> >]

In [16]:
newloop = s2.S2Loop(test1)

In [17]:
polygon = s2.S2Polygon(newloop)

In [18]:
polygon.GetArea()

2.6379255516429046e-10

In [7]:
cover = s2.S2RegionCoverer()

In [8]:
cover.set_max_level(16)
cover.set_min_level(10)

In [ ]:
cover.

In [25]:
start_time = time.time()
finalcovering = cover.GetCovering(polygon)
logger.debug("--- %s seconds ---" % (time.time() - start_time))

   --- 0.00118017196655 seconds ---


In [26]:
len(finalcovering)

8

In [27]:
[logger.debug(element.ToToken() + ', ') for element in finalcovering]

   80857c14, 
   80857c181, 
   80857c3, 
   80857c5, 
   80857c6b, 
   80857dcc, 
   80857e9c, 
   80857ea04, 


[None, None, None, None, None, None, None, None]

In [18]:
start_time = time.time()
finalcovering_loop = cover.GetCovering(loop1)
logger.debug("--- %s seconds ---" % (time.time() - start_time))

   --- 58.0674459934 seconds ---


In [19]:
start_time = time.time()
finalcovering_rect = cover.GetCovering(rectbound)
logger.debug("--- %s seconds ---" % (time.time() - start_time))

   --- 82.9485869408 seconds ---


In [28]:
len(finalcovering_loop)

6291456

In [26]:
len(finalcovering_poly)

6291456

In [31]:
first = finalcovering_poly[]

In [33]:
first.ToString()

'0/0000000000'

In [34]:
first.ToToken()

'000001'

In [18]:
rect = polygon.GetRectBound()

In [24]:
size = rect.GetCenter()

In [26]:
size.

<pywraps2.S2Point; proxy of <Swig Object of type 'S2Point *' at 0x135261ba0> >

In [42]:
polygon = s2.S2Polygon()

In [ ]:
polygon.InitNested([loop1, loop2])

In [ ]:
def addloops(alist):
    alist_copy = copy.deepcopy(alist)
    for bound in alist_copy:
        bount['loop'] = []
        if bound['type'] == 'relation':
            for waylist in bound['nodelist']:
                bound['loop'].append(s2.S2Loop())
        if bound['type'] == 'way':
            for pairindex, pair in enumerate(bound['nodelist']):
                bound['nodelist'][pairindex] = s2.S2LatLng.FromDegrees(pair[0], pair[1]).ToPoint()
    return alist_copy

In [13]:
loop = s2.S2Loop()

# stuff for fixing bad elements (too much work)

In [29]:
def listtostring(alist):
    for element in alist:
        logger.debug(str(element[0]) + ', ' + str(element[1]))

In [30]:
def testnodelist(nodelist, poplist):
    newnodelist = copy.deepcopy(nodelist)
    try:
        newnodelist[0][0][0]
    except:
        if newnodelist[0] == newnodelist[-1]:
            retain = newnodelist.pop(-1)
        newnodelist = [element for element in newnodelist if element not in poplist]
        testnodelist = copy.deepcopy(newnodelist)
        for pairindex, pair in enumerate(testnodelist):
            oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
            newpoint = oldlatlng.ToPoint().Normalize()
            testnodelist[pairindex] = newpoint
        s2.S2Loop(testnodelist)
        #apply changes if didnt crash
        newnodelist.append(retain)
        logger.debug(poplist)
        return newnodelist
    else:
        for windex, waylist in enumerate(newnodelist):
            if waylist[0] == waylist[-1]:
                retain = waylist.pop(-1)
            waylist = [element for element in waylist if element not in poplist[windex]]
            testwaylist = copy.deepcopy(waylist)
            for pairindex, pair in enumerate(testwaylist):
                oldlatlng = s2.S2LatLng.FromDegrees(float(pair[0]), float(pair[1]))
                newpoint = oldlatlng.ToPoint().Normalize()
                testwaylist[pairindex] = newpoint
            s2.S2Loop(testwaylist)
            #apply changes if didnt crash
            waylist.append(retain)
            newnodelist[windex] = waylist
        logger.debug(poplist)
        return newnodelist

In [31]:
hoveredonset = set()
def plotlist(nodelist):
    newnodelist = copy.deepcopy(nodelist)
    input_lon = np.array([pair[1] for pair in newnodelist])
    input_lat = np.array([pair[0] for pair in newnodelist])
    x, y = proj.transform(crs_wgs, crs_bng, input_lon, input_lat)
    x = (x - x.min()) / (x.max() - x.min())
    y = (y - y.min()) / (y.max() - y.min())
    
    #plt.scatter(x,y)
    
    #logger.debug(input_lon)
    
        
    tuplelist = list(newnodelist)
#     for index,element in enumerate(tuplelist):
#         logger.debug(element)
#         logger.debug(newnodelist[index])

    string_tuplelist = [str(element) for element in tuplelist]
    names = np.array(string_tuplelist)
    c = np.random.randint(1,5,size=len(newnodelist))

    norm = plt.Normalize(1,4)
    cmap = plt.cm.RdYlGn
    
    fig,ax = plt.subplots()
    sc = plt.scatter(x,y,c=c, s=30, cmap=cmap, norm=norm)

    annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                        bbox=dict(boxstyle="round", fc="w"),
                        arrowprops=dict(facecolor='black', shrink=0.05),horizontalalignment='center')
    annot.set_visible(False)

    def update_annot(ind):

        pos = sc.get_offsets()[ind["ind"][0]]
        annot.xy = pos
        text = "{}, {}".format(" ".join(list(map(str,ind["ind"]))), 
                               " ".join([names[n] for n in ind["ind"]]))
        annot.set_text(text)
        annot.get_bbox_patch().set_facecolor(cmap(norm(c[ind["ind"][0]])))
        annot.get_bbox_patch().set_alpha(0.4)


    def hover(event):
        vis = annot.get_visible()
        if event.inaxes == ax:
            cont, ind = sc.contains(event)
            if cont:
                for n in ind["ind"]:
                    hoveredonset.add(tuple(tuplelist[n]))
                update_annot(ind)
                annot.set_visible(True)
                fig.canvas.draw_idle()
            else:
                if vis:
                    annot.set_visible(False)
                    fig.canvas.draw_idle()

    fig.canvas.mpl_connect("motion_notify_event", hover)
    

    plt.show()

In [ ]:
plotlist(bounding_university[1812]['nodelist'])

In [12]:
hoveredonset

set()

In [ ]:
newtest = testnodelist(bounding_university[1812]['nodelist'], map(lambda x: list(x), list(hoveredonset)))

In [11]:
forbu514 = [[-21.3412867, 55.491437], [-21.3411172, 55.4916237], [-21.3410765, 55.4915734], [-21.3409724, 55.49143], [-21.34091, 55.4913373], [-21.340846, 55.4912573], [-21.3407704, 55.4912861], [-21.3408304, 55.4913315], [-21.340844, 55.4913683], [-21.3408617, 55.4913881], [-21.3409271, 55.491402], [-21.3410451, 55.4914876], [-21.3409843, 55.4915411], [-21.3410342, 55.4915717], [-21.3410612, 55.4916359], [-21.3410142, 55.4917124], [-21.3409273, 55.4917522], [-21.3408475, 55.4917675], [-21.3407748, 55.491769], [-21.3407321, 55.4917935], [-21.3406964, 55.4917782], [-21.3406708, 55.4917828], [-21.3406252, 55.4917966], [-21.3405853, 55.4917706], [-21.3405454, 55.4917415], [-21.3404827, 55.4917354], [-21.3404528, 55.4917293], [-21.3404285, 55.4917415], [-21.3404186, 55.4917216], [-21.3404186, 55.4916788], [-21.3404247, 55.4915998], [-21.3404889, 55.4915656], [-21.3405425, 55.4915747], [-21.340571, 55.4914842], [-21.3406466, 55.4914814], [-21.3406352, 55.491434], [-21.3406537, 55.4913697], [-21.3407025, 55.4913045], [-21.3403466, 55.4914245], [-21.3403259, 55.4913611], [-21.3403396, 55.4913407], [-21.3403847, 55.4914644], [-21.3403496, 55.491544]]
forbu1296 = [[[48.8186574, 2.3328606],[48.8219296, 2.3305399],[48.8219326, 2.3305229]], [[48.8185842, 2.3330875]]]
forbu1378 = [[53.3467438, -6.2575819], [53.3467719, -6.2576414], [53.3467418, -6.2576384], [53.3467831, -6.2574094], [53.3467757, -6.2572767], [53.3467739, -6.257584], [53.3467683, -6.2574077], [53.346714, -6.2572612], [53.3465712, -6.2574273], [53.3467534, -6.2574061], [53.3466603, -6.2578816], [53.3467584, -6.2580085], [53.3465556, -6.2577282], [53.3467294, -6.2578878]]

In [12]:
buredodict = {514: forbu514, 1296: forbu1296, 1378: forbu1378}
bcredodict = {}
blredodict = {}

In [13]:
def cleandata(selectionlist, repairdict):
    for index, value in repairdict.items():
        selectionlist[index]['nodelist'] = testnodelist(selectionlist[index]['nodelist'], value)
    return selectionlist

In [14]:
bounding_university = cleandata(bounding_university,buredodict)
bounding_college = cleandata(bounding_college,bcredodict)
bounding_local = cleandata(bounding_local,blredodict)

   [[[48.8186574, 2.3328606], [48.8219296, 2.3305399], [48.8219326, 2.3305229]], [[48.8185842, 2.3330875]]]
   [[-21.3412867, 55.491437], [-21.3411172, 55.4916237], [-21.3410765, 55.4915734], [-21.3409724, 55.49143], [-21.34091, 55.4913373], [-21.340846, 55.4912573], [-21.3407704, 55.4912861], [-21.3408304, 55.4913315], [-21.340844, 55.4913683], [-21.3408617, 55.4913881], [-21.3409271, 55.491402], [-21.3410451, 55.4914876], [-21.3409843, 55.4915411], [-21.3410342, 55.4915717], [-21.3410612, 55.4916359], [-21.3410142, 55.4917124], [-21.3409273, 55.4917522], [-21.3408475, 55.4917675], [-21.3407748, 55.491769], [-21.3407321, 55.4917935], [-21.3406964, 55.4917782], [-21.3406708, 55.4917828], [-21.3406252, 55.4917966], [-21.3405853, 55.4917706], [-21.3405454, 55.4917415], [-21.3404827, 55.4917354], [-21.3404528, 55.4917293], [-21.3404285, 55.4917415], [-21.3404186, 55.4917216], [-21.3404186, 55.4916788], [-21.3404247, 55.4915998], [-21.3404889, 55.4915656], [-21.3405425, 55.4915747], [-21.3